In [12]:
#Task 4.1
import sqlite3
from sqlite3 import Error

def db_connect(filename):
    try:
        conn = sqlite3.connect(filename)
        return conn
    except Error as e:
        print(e)
        
def db_create_table(conn, sql):
    try:
        conn.execute(sql)
    except Error as e:
        print(e)

students_sql = '''
               CREATE TABLE students (
                   student_id INTEGER,
                   stu_name TEXT NOT NULL,
                   gender TEXT NOT NULL,
                   primary_sch TEXT NOT NULL,
                   guardian_id INTEGER NOT NULL,
                   PRIMARY KEY(student_id),
                   FOREIGN KEY(guardian_id) REFERENCES guardians(guardian_id),
                   CHECK ( gender = 'male' OR gender = 'female')
                   );
               '''

guardians_sql = '''
               CREATE TABLE guardians (
                   guardian_id INTEGER,
                   guardian_name TEXT NOT NULL,
                   guardian_email TEXT NOT NULL,
                   PRIMARY KEY(guardian_id)
                   )
               '''
results_sql = '''
               CREATE TABLE results (
                   student_id INTEGER,
                   subject TEXT NOT NULL,
                   al INTEGER NOT NULL,
                   PRIMARY KEY(student_id, subject),
                   FOREIGN KEY(student_id) REFERENCES students(student_id),
                   CHECK ( al > 0 AND al < 9)
                   );
               ''' 
table_sql = [guardians_sql, students_sql, results_sql ]

db_conn = db_connect("ENROLMENT.db")
for sql in table_sql:
    db_create_table(db_conn, sql)
db_conn.close()

table guardians already exists
table students already exists
table results already exists


In [13]:
#Task 4.2
import csv
filename1 = 'GUARDIAN.csv'
filename2 = 'STUDENT.csv'
filename3 = 'RESULT.csv'

sql1 = '''
       INSERT INTO guardians(guardian_id, guardian_name, guardian_email)
           VALUES(?,?,?);
       '''
sql2 = '''
       INSERT INTO students(student_id, stu_name, gender, primary_sch, guardian_id)
           VALUES(?,?,?,?,?);
       '''
sql3 = '''
       INSERT INTO results(student_id, subject, al)
           VALUES(?,?,?);
       '''

db_conn = db_connect("ENROLMENT.db")
import_seq = [[filename1, sql1],[filename2, sql2], [filename3, sql3]]
for filename, sql in import_seq:
    with open(filename,'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        for row in csvreader:
            try:
                db_conn.execute(sql, row)
                db_conn.commit()
            except Error as e:
                print(e)
db_conn.close()

In [14]:
#Task 4.3
def update_guardian_email(conn, guardian_id, new_email):
    sql = '''
          UPDATE guardians
              SET guardian_email = ?
              WHERE guardian_id = ?
          '''
    try:
        conn.execute(sql, (new_email, guardian_id))
        conn.commit()
        print("update completed")
    except Error as e:
        print(e)
    
db_conn = db_connect("ENROLMENT.db")
update_guardian_email(db_conn, 900005, "see@see.com")
db_conn.close()

update completed


In [15]:
#Task 4.4 Web app page 1
def pri_sch_summary(conn):
    sql = '''
          SELECT primary_sch, count(*)/4 as 'no of student', round(sum(al)*1.0/count(*)*4,2) as 'mean overall AL' FROM results
              JOIN students
              ON results.student_id = students.student_id
              GROUP by primary_sch
              ORDER BY primary_sch;
          '''
    cur = conn.execute(sql)
    rows = cur.fetchall()
    return list(rows)
db_conn = db_connect("ENROLMENT.db")
print(pri_sch_summary(db_conn))
db_conn.close()

[('Boon Day Park Primary', 4, 9.25), ('Lurang Primary', 4, 10.75), ('Shuqu Primary', 3, 8.33)]


In [16]:
#Task 4.5 web app page 2
def get_students(conn, cutoff):
    
    sql = '''SELECT stu_name, primary_sch, sum(al) as overall_al, guardian_name, guardian_email FROM results
                 LEFT JOIN students
                     ON students.student_id = results.student_id
                 LEFT JOIN guardians
                     ON students.guardian_id = guardians.guardian_id
                 GROUP BY results.student_id
                 HAVING overall_al < ?
                 ORDER BY primary_sch, overall_al
          '''
    cur = conn.execute(sql,(cutoff,))
    rows = cur.fetchall()
    return list(rows)
db_conn = db_connect("ENROLMENT.db")
print(get_students(db_conn, 10))
db_conn.close()

[('Python Gopal', 'Boon Day Park Primary', 7, 'Gopal', 'gopal_fam@kmail.com'), ('JinJa Oh', 'Boon Day Park Primary', 8, 'Oh', 'oh_fam@kmail.com'), ('Sequl Lite', 'Lurang Primary', 8, 'Lite', 'lite_fam@kmail.com'), ('Perl Tan', 'Lurang Primary', 9, 'Tan', 'tan_fam@imail.com'), ('See Sharp', 'Shuqu Primary', 7, 'See', 'see@see.com'), ('Flask Chan', 'Shuqu Primary', 9, 'Chan', 'chan_fam@imail.com'), ('Mac Ahmad', 'Shuqu Primary', 9, 'Ahmad', 'ahmad_fam@kmail.com')]
